In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from polygon import RESTClient
import datetime as dt

import factor_analysis_functions
import stock_data_functions
import ewma_beta_tuning
from matplotlib.dates import MonthLocator, DateFormatter
from stock_data_functions import TickerComparison
from typing import List, Optional, Dict, Any, Tuple
import seaborn as sns

import importlib
importlib.reload(stock_data_functions)
importlib.reload(factor_analysis_functions)
importlib.reload(ewma_beta_tuning)


client = RESTClient('tt2gOLH0fHAmPX70a4QURLFy59PRCZr3')
API_key = 'tt2gOLH0fHAmPX70a4QURLFy59PRCZr3'


import scipy.stats as stats
from fredapi import Fred
fred_api = Fred('e48d0413b1cd0a3b30b58d42225373de')

In [2]:
from read_in_data_functions import get_sub_sectors_tickers, SUB_SECTOR_DICT

cut_off_cap = 10_000 
tech_stocks_marketcap = get_sub_sectors_tickers(sector='Technology', n = 60) 
tech_sectors          = list(tech_stocks_marketcap.columns.get_level_values(0).unique())

In [3]:
mask = (tech_stocks_marketcap[tech_stocks_marketcap.loc[:,(slice(None), 'Market Cap')] > cut_off_cap] > 0)
tech_stocks_marketcap = tech_stocks_marketcap.loc[mask.any(axis=1), :]

long = (
    tech_stocks_marketcap
    .stack(level=0, future_stack=True)                       # index: (row, sector); columns: ['Symbol','Market Cap']
    .dropna(subset=['Market Cap'])        # keep rows with a market cap
)

result = (
    long
    .groupby(level=1)                     # group by sector (the stacked level)
    .agg(avg_market_cap=('Market Cap', lambda x : x.mean().round(2)),  # average market cap per sector
         n_stocks=('Symbol', 'count'))    # counts non-null symbols
    .sort_index()
)

result.sort_values(by='avg_market_cap', ascending=False)


,avg_market_cap,n_stocks
semiconductors,260681.35,37
consumer-electronics,244853.35,17
software-infrastructure,162241.08,37
software-application,67046.22,37
semiconductor-equipment-and-materials,35560.35,30
information-technology-services,23775.68,37
computer-hardware,17908.78,32
communication-equipment,17162.65,37
electronic-components,13168.40,37
scientific-and-technical-instruments,6504.18,31


In [12]:
semi_tickers           = tech_stocks_marketcap['semiconductors']['Symbol']
semi_equipment_tickers = tech_stocks_marketcap['semiconductor-equipment-and-materials']['Symbol']
software_tickers       = tech_stocks_marketcap['software-application']['Symbol']
computer_hardware_tickers = tech_stocks_marketcap['computer-hardware']['Symbol']
comm_equipment_tickers = tech_stocks_marketcap['communication-equipment']['Symbol']
software_infrastructure_tickers = tech_stocks_marketcap['software-infrastructure']['Symbol']
electronic_components_tickers = tech_stocks_marketcap['electronic-components']['Symbol']
scientific_technical_instruments_tickers = tech_stocks_marketcap['scientific-and-technical-instruments']['Symbol']

all_tickers = {#'semi' : semi_tickers,
               'semi_equipment' : semi_equipment_tickers,
               'software' : software_tickers, 'computer_hardware' : computer_hardware_tickers,
               'comm_equipment' : comm_equipment_tickers,
               'software_infrastructure' : software_infrastructure_tickers,
               'electronic_components' : electronic_components_tickers,
               'scientific_technical_instruments' : scientific_technical_instruments_tickers
              }

In [ ]:
ticker_objs = {}
for sector, tickers in all_tickers.items():
    tC_min = TickerComparison(tickers, period='minute', filing_date_gte='2023-11-06', fetch_in_chunks = True, date_updated=True, end_date='2025-11-04', waiting_time=60, chunksize=500)
    tC_day = TickerComparison(tickers, period='day',filing_date_gte='2023-11-06',  fetch_in_chunks = True, date_updated=True, end_date='2025-11-04', waiting_time=60, chunksize = 500)
    ticker_objs[sector] = [tC_min, tC_day]

1. Regress daily close to close -> stock returns against different index (3)
2. Regression is EWMA WLS 
3. Calculate R2 on 5 minute returns 
4. Use as regime
5. Calculate forward returns based on regime x ticker
6. Add in VIX, term spread ...